
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Climate emulator output generation

inputs required: 
  * historical wave and water level conditions
  * emulator output - synthetic wave and water level conditions
  
in this notebook:
  * plot the historical and synthetic values of the analyzed variables
  * generation of hourly values of the synthetic time series
  
  


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.climate_emulator import Climate_Emulator
from teslakit.waves import AWL, Aggregate_WavesFamilies, Intradaily_Hydrograph
from teslakit.util.time_operations import xds_limit_dates, repair_times_hourly, add_max_storms_mask

from teslakit.plotting.outputs import Plot_Complete



## Database and Site parameters

In [ ]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')

# climate emulator object
CE = Climate_Emulator(db.paths.site.EXTREMES.climate_emulator)
CE.Load()

n_sims_DWTs = 10



## Historical Waves and Climate Data

In [ ]:
# generate historical complete hourly data 
HIST_C_h = db.Generate_HIST_Complete()

# aggregate waves families
HIST_C_h = HIST_C_h.drop(['Hs', 'Tp', 'Dir'])
WVS_a = Aggregate_WavesFamilies(HIST_C_h)
HIST_C_h = xr.merge([HIST_C_h, WVS_a])

# set AT reference level (mean at 0)
HIST_C_h['AT'] = HIST_C_h['AT'] - np.nanmean(HIST_C_h['AT'])

# calculate AWL, TWL and level
HIST_C_h['AWL'] = AWL(HIST_C_h['Hs'], HIST_C_h['Tp'])
HIST_C_h['TWL'] = HIST_C_h['AWL'] + HIST_C_h['AT'] + HIST_C_h['MMSL']
HIST_C_h['level'] = HIST_C_h['AT'] + HIST_C_h['MMSL']

# repair times: round to hour and remove duplicates (if any)
HIST_C_h = repair_times_hourly(HIST_C_h)

# add mask for max_storms times
HIST_C_h = add_max_storms_mask(HIST_C_h, CE.WVS_MS.time.values)

# store hourly historical data
db.Save_HIST_Complete_hourly(HIST_C_h)

print(HIST_C_h)


In [ ]:
# Plot Historical 

Plot_Complete(HIST_C_h);



## Simulated Waves and Climate Data

In [ ]:
# --------------------------------------
# Climate Emulator extremes model output

def waves_hourly(CE, n_sims_DWTs, n_sim_ce=0):
    '''
    Generate hourly simulation output. 
    
    - join DWTs-WVs Climate emulator simulations for all n_sims_DWTs
    - aggregate simulated waves storms
    - generate hourly hydrographs
    
    return xarray.Dataset with dims: n_sim (DWTs-WVs), time (hourly)
    '''

    # iterate simulations
    ls_hy = []
    for n in range(n_sims_DWTs):
        _, TCS_sim, WVS_upd = CE.LoadSim(n_sim = n)    
        
        # select one Climate Emulator sim. for each DWT sim.
        WVS_upd = WVS_upd.isel(n_sim = n_sim_ce)  
        TCS_sim = TCS_sim.isel(n_sim = n_sim_ce)
        
        # aggregate waves families
        wvs_agr = Aggregate_WavesFamilies(WVS_upd)
        
        # calculate intradaily hourly hydrographs for simulated storms
        hy = Intradaily_Hydrograph(wvs_agr, TCS_sim)
        
        # repair times: remove duplicates (if any)
        hy = repair_times_hourly(hy)

        # add mask for max_storms times
        hy = add_max_storms_mask(hy, WVS_upd.time.values)
        
        ls_hy.append(hy)
        
    # combine hourly simulations
    hy = xr.concat(ls_hy, 'n_sim')

    return hy


# generate hourly waves using intradaily hydrographs
WVS_sim_h = waves_hourly(CE, n_sims_DWTs)  

# store hourly simulated waves data
db.Save_SIM_Waves_hourly(WVS_sim_h)

print(WVS_sim_h)



## Complete Simulation: load covariates and merge with hourly waves

In [ ]:
# --------------------------------------
# Covariates: AWTs, DWTs, MJO, MMSL, AT

# generate hourly covariates data
CVS_sim_h = db.Generate_SIM_Covariates(total_sims=n_sims_DWTs)

# set AT reference level (mean at 0)
for s in CVS_sim_h.n_sim:
    CVS_sim_h.sel(n_sim=s)['AT'].values[:] = CVS_sim_h.sel(n_sim=s)['AT'] - np.nanmean(CVS_sim_h.sel(n_sim=s)['AT'])

# store hourly covariates
db.Save_SIM_Covariates_hourly(CVS_sim_h)


In [ ]:
# --------------------------------------
# Complete simulation output 

d1, d2 = xds_limit_dates([WVS_sim_h, CVS_sim_h])
WVS_sim_h = WVS_sim_h.sel(time = slice(d1, d2))
CVS_sim_h = CVS_sim_h.sel(time = slice(d1, d2))

# merge all data
SIM_C_h = xr.merge([WVS_sim_h, CVS_sim_h])

# calculate AWL, TWL and level
SIM_C_h['AWL'] = AWL(SIM_C_h['Hs'], SIM_C_h['Tp'])
SIM_C_h['TWL'] = SIM_C_h['AWL'] + SIM_C_h['SS'] + SIM_C_h['AT'] + SIM_C_h['MMSL']
SIM_C_h['level'] = SIM_C_h['SS'] + SIM_C_h['AT'] + SIM_C_h['MMSL']

# store hourly complete data
db.Save_SIM_Complete_hourly(SIM_C_h)

print(SIM_C_h)


In [ ]:
# Plot one Simulation 

Plot_Complete(SIM_C_h.sel(n_sim=0));
